## Установка всех необходимых библиотек

# Importing needed libraries

In [1]:
import openai
from TTS.api import TTS
import os
from pydub import AudioSegment

c:\Users\HP\Desktop\Outpeer\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\HP\Desktop\Outpeer\.venv\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


# OpenAI API Key (unique for every account)

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

# Background Music

In [15]:
# 1) Background music selection
# --------------------------------
# Map certain theme keywords to audio files
THEME_MUSIC_MAP = {
    "space": "C:\\Users\\HP\\Desktop\\Outpeer\\NLP_Project\\background_sound\\space.wav",
    "fantastic": "C:\\Users\\HP\\Desktop\\Outpeer\\NLP_Project\\background_sound\\fantasy.wav",
    "medieval": "C:\\Users\\HP\\Desktop\\Outpeer\\NLP_Project\\background_sound\\medieval.wav",
    "horror": "C:\\Users\\HP\\Desktop\\Outpeer\\NLP_Project\\background_sound\\horror.wav",
    "sea": "C:\\Users\\HP\\Desktop\\Outpeer\\NLP_Project\\background_sound\\sea.wav",
    "sci-fi": "C:\\Users\\HP\\Desktop\\Outpeer\\NLP_Project\\background_sound\\sci-fi.wav",
    "forest": "C:\\Users\\HP\\Desktop\\Outpeer\\NLP_Project\\background_sound\\day_forest.wav"
    }

DEFAULT_BG_MUSIC = "C:\\Users\\HP\\Desktop\\Outpeer\\NLP_Project\\background_sound\\fairy_tail_slow.wav"  # fallback if no matching theme


# Background Music Selection

In [16]:
def pick_background_music(themes):
    """
    Given the user-input themes (string),
    determine which background music file to use.

    If multiple themes match, we'll pick the first one we find.
    If none match, return the default.
    """
    # Make the theme input lowercase for matching
    themes_lower = themes.lower()

    for key, music_file in THEME_MUSIC_MAP.items():
        if key in themes_lower:
            return music_file

    # If no matches, return default music
    return DEFAULT_BG_MUSIC

# Fairy Tale Generation

In [17]:
# 2) Fairy tale generation
# --------------------------------
def generate_fairy_tale():
    """
    Prompts the user for custom input, then uses OpenAI ChatCompletion
    to generate a fairy tale in Russian.
    Returns the fairy tale text (string) and user themes.
    """
    user_prompt = input("Enter an initial scenario or prompt for your fairy tale: ")
    main_character = input("Who is your main character? ")
    themes = input("List the themes (e.g., space, medieval, fantastic, sci-fi): ")

    system_message = (
        "You are a creative AI specialized in crafting original fairy tales. "
        "Write a complete story with a clear beginning, middle, and end. It should be relatively small story which includes 1000 words. "
        "Use rich detail, do not end abruptly, and conclude with a final resolution. Change to Russian language."
    )
    user_message = (
        f"Initial Scenario: {user_prompt}\n"
        f"Main Character: {main_character}\n"
        f"Themes: {themes}\n\n"
        "Please write the fairy tale using these elements, up to around 1000 words, ending conclusively."
    )

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]

    try:
        response = openai.chat.completions.create(
            model = "gpt-4o-mini",
            messages = messages,
            temperature = 0.7,
            max_tokens = 10,    # For testing; increase if you need a longer story
            top_p = 1.0,
            frequency_penalty = 0.0,
            presence_penalty = 0.0
        )

        fairy_tale = response.choices[0].message.content
        return fairy_tale, themes

    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

# Text to Speech Change using Coqui (Self-Generated Voice)

In [18]:
# 3) TTS synthesis (Coqui)
# --------------------------------
def text_to_speech_coqui(story_text, output_path="fairy_tale.wav"):
    """
    Uses a Coqui TTS model to synthesize speech from the given text.
    Saves the audio to 'fairy_tale.wav' by default.
    """
    try:
        # Choose a pretrained TTS model
        model_name = "tts_models/multilingual/multi-dataset/xtts_v2"
        tts = TTS(model_name=model_name)

        # Convert text to an audio file
        # For advanced usage, you might specify "speaker" if needed
        # or "speaker_wav" for voice cloning, etc.
        tts.tts_to_file(
            text=story_text,
            speaker_wav = "Islam.wav",
            file_path=output_path,
            language='ru'
        )

        print(f"Fairy tale narration saved to '{output_path}'.")
        return output_path

    except Exception as e:
        print(f"An error occurred during TTS: {e}")
        return None



# Adding Background Music to Generated Story

In [19]:
# 4) Mixing narration + background
# --------------------------------
def mix_audio_files(narration_path, background_path, output_path, music_volume_db=-15, fade_out_ms=3000):
    """
    Overlay the narration on top of background music:
    1. Truncate/fade out background music to match narration length.
    2. Lower background volume.
    3. Save the mixed result as output_path.

    :param narration_path: Path to the .wav file with TTS narration.
    :param background_path: Path to the .wav file with background music.
    :param output_path: Path to output the mixed audio.
    :param music_volume_db: Amount to lower background volume (in dB).
    :param fade_out_ms: Milliseconds of fade out on the background music at the end.
    """
    # Load the narration
    narration = AudioSegment.from_file(narration_path)
    narration_duration = len(narration)  # in ms

    # Load the background music
    background = AudioSegment.from_file(background_path)

    # Trim or repeat background to match (or exceed) the narration duration
    # If music is shorter than narration, loop it or you can do something else:
    if len(background) < narration_duration:
        # Simple approach: loop the background until it matches or exceeds narration length
        times_to_repeat = (narration_duration // len(background)) + 1
        background = background * times_to_repeat

    # Now fade out the last part of the background so it ends smoothly
    background = background[:narration_duration].fade_out(fade_out_ms)

    # Lower background volume
    background = background + music_volume_db  # e.g., -10 dB

    # Overlay the narration on top of the background
    # The 'overlay' starts both tracks at the same time = 0ms
    final_mix = background.overlay(narration, position=0)

    # Export the final track
    final_mix.export(output_path, format="wav")
    print(f"Final mixed audio saved to '{output_path}'.")


# Main

In [20]:
# 5) Main workflow
# --------------------------------
def main():
    # 1. Generate the story + capture the user’s theme(s)
    fairy_tale, themes = generate_fairy_tale()
    if fairy_tale is None:
        return

    print("\n=== AI-Generated Fairy Tale (Russian) ===\n")
    print(fairy_tale)

    # 2. Pick background music based on user themes
    if themes is None:
        bg_music = DEFAULT_BG_MUSIC
    else:
        bg_music = pick_background_music(themes)

    # 3. Convert the generated story to speech
    narration_file = text_to_speech_coqui(fairy_tale, output_path="fairy_tale.wav")
    if narration_file is None:
        return

    # 4. Mix TTS narration with background music
    final_output = "final_fairy_tale_mix.wav"
    mix_audio_files(
        narration_path=narration_file,
        background_path=bg_music,
        output_path=final_output,
        music_volume_db=-15,    # reduce music volume by 15 dB
        fade_out_ms=3000        # 3 second fade at the end
    )

    print("\n=== Done! ===")
    print(f"Your final narrated fairy tale with background music is: {final_output}")

if __name__ == "__main__":
    main()


=== AI-Generated Fairy Tale (Russian) ===

В далеком королевстве, спрятан
Fairy tale narration saved to 'fairy_tale.wav'.
Final mixed audio saved to 'final_fairy_tale_mix.wav'.

=== Done! ===
Your final narrated fairy tale with background music is: final_fairy_tale_mix.wav
